In [1]:
import numpy as np
import torch 
import torch.nn as nn
from torch.autograd import Variable
import sys
sys.path.append('..')
from function.CDSSM import CDSSM

In [2]:
# 读取字典
char2id = {}
id2char = {}
with open("../../../../../resources/corpus/lexical/char_7000.txt", mode='r', encoding='utf-8') as f:
    lines = f.readlines()
    for idx, vocab in enumerate(lines):
        vocab = vocab.strip()
        char2id[vocab] = idx 
        id2char[idx] = vocab 


In [4]:
# 中文数据 构造方式 1 三体 + ES
# l_Qs 为关于三体的相关问题
# pos_Ds 为使用es 检索得到的结果
# neg_Ds 为随机得到的结果（每次的随机种子固定）

In [5]:
# 中文数据 构造方式2  CMRC2018 等数据构造
# l_Qs 数据集中question
# pos_Ds 数据集中的answer
# neg_Ds 数据集中的其他answer

train_question = []
train_true_answer = []
train_false_answer = []
all_document = []

with open('../../../../../resources/corpus/qg/cmrc2018/train_pa_list.txt', mode='r', encoding='utf-8') as f:
    train_pa_lines = f.readlines()
with open('../../../../../resources/corpus/qg/cmrc2018/train_q_list.txt', mode='r', encoding='utf-8') as f:
    train_q_lines = f.readlines()

for line in train_pa_lines:
    line = line.strip()
    if len(line.split('[SEP]')) !=2:
        continue
    item1, item2 = line.split('[SEP]')
    train_true_answer.append(item1)
    all_document.append(item1)

for line in train_q_lines:
    train_question.append(line.strip())



In [6]:
print(train_question[:1])
print(train_true_answer[:1])
print(all_document[:1])

['范 廷 颂 是 什 么 时 候 被 任 为 主 教 的 ？']
['范 廷 颂 枢 机 （ ， ） ， 圣 名 保 禄 · 若 瑟 （ ） ， 是 越 南 罗 马 天 主 教 枢 机 。 1 9 6 3 年 被 任 为 主 教 ； 1 9 9 0 年 被 擢 升 为 天 主 教 河 内 总 教 区 宗 座 署 理 ； 1 9 9 4 年 被 擢 升 为 总 主 教 ， 同 年 年 底 被 擢 升 为 枢 机 ； 2 0 0 9 年 2 月 离 世 。 范 廷 颂 于 1 9 1 9 年 6 月 1 5 日 在 越 南 宁 平 省 天 主 教 发 艳 教 区 出 生 ； 童 年 时 接 受 良 好 教 育 后 ， 被 一 位 越 南 神 父 带 到 河 内 继 续 其 学 业 。 范 廷 颂 于 1 9 4 0 年 在 河 内 大 修 道 院 完 成 神 学 学 业 。 范 廷 颂 于 1 9 4 9 年 6 月 6 日 在 河 内 的 主 教 座 堂 晋 铎 ； 及 后 被 派 到 圣 女 小 德 兰 孤 儿 院 服 务 。 1 9 5 0 年 代 ， 范 廷 颂 在 河 内 堂 区 创 建 移 民 接 待 中 心 以 收 容 到 河 内 避 战 的 难 民 。 1 9 5 4 年 ， 法 越 战 争 结 束 ， 越 南 民 主 共 和 国 建 都 河 内 ， 当 时 很 多 天 主 教 神 职 人 员 逃 至 越 南 的 南 方 ， 但 范 廷 颂 仍 然 留 在 河 内 。 翌 年 管 理 圣 若 望 小 修 院 ； 惟 在 1 9 6 0 年 因 捍 卫 修 院 的 自 由 、 自 治 及 拒 绝 政 府 在 修 院 设 政 治 课 的 要 求 而 被 捕 。 1 9 6 3 年 4 月 5 日 ， 教 宗 任 命 范 廷 颂 为 天 主 教 北 宁 教 区 主 教 ， 同 年 8 月 1 5 日 就 任 ； 其 牧 铭 为 「 我 信 天 主 的 爱 」 。 由 于 范 廷 颂 被 越 南 政 府 软 禁 差 不 多 3 0 年 ， 因 此 他 无 法 到 所 属 堂 区 进 行 牧 灵 工 作 而 专 注 研 读 等 工 作 。 范 廷 颂 除 了 面 对 战 争 、 贫 困 、 被 当 局 迫 害 天 主 教 会 等 问 题 外 ， 也 秘 密 恢 

In [ ]:
# 中文数据预处理
# 获得 l_Qs, pos_l_Ds, neg_l_Ds

# sample_size = 10
# l_Qs = []
# pos_l_Ds = []

# (query_len, doc_len) = (5, 100)

# for i in range(sample_size):
#     query_len = np.random.randint(1, 10)
#     l_Q = np.random.rand(1, query_len, WORD_DEPTH)
#     l_Qs.append(l_Q)
    
#     doc_len = np.random.randint(50, 500)
#     l_D = np.random.rand(1, doc_len, WORD_DEPTH)
#     pos_l_Ds.append(l_D)

# neg_l_Ds = [[] for j in range(J)]
# for i in range(sample_size):
#     possibilities = list(range(sample_size))
#     possibilities.remove(i)
#     negatives = np.random.choice(possibilities, J, replace = False)
#     for j in range(J):
#         negative = negatives[j]
#         neg_l_Ds[j].append(pos_l_Ds[negative])

# # Till now, we have made a complete numpy dataset
# # Now let's convert the numpy variables to torch Variable

# for i in range(len(l_Qs)):
#     l_Qs[i] = Variable(torch.from_numpy(l_Qs[i]).float())
#     pos_l_Ds[i] = Variable(torch.from_numpy(pos_l_Ds[i]).float())
#     for j in range(J):
#         neg_l_Ds[j][i] = Variable(torch.from_numpy(neg_l_Ds[j][i]).float())

In [ ]:
LETTER_GRAM_SIZE = 3 # See section 3.2.
WINDOW_SIZE = 3 # See section 3.2.
TOTAL_LETTER_GRAMS = int(3 * 1e4) # Determined from data. See section 3.2.
WORD_DEPTH = WINDOW_SIZE * TOTAL_LETTER_GRAMS # See equation (1).
# Uncomment it, if testing
# WORD_DEPTH = 1000
K = 300 # Dimensionality of the max-pooling layer. See section 3.4.
L = 128 # Dimensionality of latent semantic space. See section 3.5.
J = 4 # Number of random unclicked documents serving as negative examples for a query. See section 4.
FILTER_LENGTH = 1 # We only consider one time step for convolutions.

In [ ]:
model = CDSSM()

# Build a random data set.
import numpy as np


# Loss and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)

# output variable, remember the cosine similarity with positive doc was at 0th index
y = np.ndarray(1)
# CrossEntropyLoss expects only the index as a long tensor
y[0] = 0
y = Variable(torch.from_numpy(y).long())

for i in range(sample_size):
    y_pred = model(l_Qs[i], pos_l_Ds[i], [neg_l_Ds[j][i] for j in range(J)])
    print(y_pred)
    loss = criterion(y_pred.resize(1,J+1), y)
    # print (i, loss.data[0])
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
# 